# Import Libraries

In [1]:
import requests
import csv
from parsel import Selector
import os
import random

# Scraping Function

#### Note: I've supposed that the category is Audio & Hifi in Electronics, is page: https://www.bol.com/nl/nl/sf/electronics/ there are many categories (Audio & Hifi) is one of them 

In [2]:
def main(page):
    select = Selector(text=page.text)
    
    #list to store dictionaries containing the product information
    prod_list = []
    
    #selects a list of products on the web page
    products = select.xpath('//*[@id="js_items_content"]/li')
    print("Number Of Products = ", len(products))
    
    #extracts information from the product
    def get_info(product):
        pid = product.xpath('@data-id').extract_first()
        #print(pid)
        
        title = product.xpath('.//a[contains(@class, "product-title")]/text()').extract_first()
        #print(title)
        
        brand = product.xpath('.//a[@data-test="party-link"]/text()').extract_first()
        #print(brand)
        
        #to handle errors
        try:
            rate = product.xpath('.//div[@class="u-mb--xs"]/@aria-label').extract_first().split()[0]
            #print(rate)
        except:
            rate = ""
        
        n_reviews = product.xpath('.//div[@class="star-rating"]/@data-count').extract_first()
        #print(n_reviews)
        
        url = "https://www.bol.com" + product.xpath('.//a[contains(@class, "product-title")]/@href').extract_first()
        #print(url)
        
        #to handle errors
        try:
            # Extract the price and price fraction
            price_elem = product.xpath('.//span[contains(@class, "promo-price")]/text()').extract_first().strip()
            price_fraction_elem = product.xpath('.//sup[contains(@class, "promo-price__fraction")]/text()').extract_first().strip()

            if price_fraction_elem == "-":
                price_fraction_elem = "00"

            #Concatenate the price and price fraction
            price = price_elem + ',' + price_fraction_elem if price_fraction_elem else price_elem
            #print(price)
        except:
            price=""
        
        category = select.xpath('//h1[contains(@class, "bol_header")]/text()').extract_first()
        #print(category)
        
        prod_list.append({"pid": pid, "title": title, "brand": brand, "rating": rate, "n_reviews":n_reviews, "url": url, "price": price, "category_name": category})
        
    #iterates over each product in the products list
    for product in products:
        get_info(product)
        
    #the path the file will be saved in
    path = "Products_Data.csv"
    
    #to check if the file is exists or not
    file_exists = os.path.isfile(path)
    
    #to extract the header
    fieldnames = prod_list[0].keys()

    with open(path, "a", errors='ignore', newline='') as file: #"a" for append
        writer = csv.DictWriter(file, fieldnames)

        #to write the header one time
        if not file_exists:
            writer.writeheader()

        writer.writerows(prod_list)    

# Function To Handle which page to scrape

In [3]:
def scrape(url, num):
    if num == 1:
        page = requests.get(url, proxies=proxies)
    else:
        page = requests.get(url + "?page=" + str(num))

    main(page)
    print("Done")

In [12]:
page_num = input("Enter The Number of Page you want to scrape data from: ")
url = "https://www.bol.com/nl/nl/l/audio-hifi/10714/"

scrape(url, page_num)

Enter The Number of Page you want to scrape data from: 400
Number Of Products =  24
Done
